In [ ]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("PINECONE")

In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [7]:
from pinecone import Pinecone,ServerlessSpec
index_name = 'hybrid-search-langchian-pinecone'
pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, ## dim of dense vector
        metric='dotproduct', ## sparse values supported only for dotproduct
        spec = ServerlessSpec(cloud='aws',region='us-east-1'),
    )

In [8]:
index = pc.Index(index_name)
index

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [ ]:
sentences = [
    "In 2024, I visited Paris",
    "In 2023, I visited Japan",
    "In 2022, I visited Korea"
]

#tfidf values on these sentences
bm25_encoder.fit(sentences)

bm25_encoder.dump("bm25_values.json")

bm25_encoder = BM25Encoder().load('bm25_values.json')

In [ ]:
retriever = PineconeHybridSearchRetriever(embedding,sparse_encoder=bm25_encoder,index=index)
retriever

In [ ]:
retriever.add_texts([
    "In 2024, I visited Paris",
    "In 2023, I visited Japan",
    "In 2022, I visited Korea"
])

In [ ]:
retriever.invoke("What city did i visit last")